# Flaw Engine: Cloud Trainer (Gideon-gg101 Edition) 🚀

This notebook turns this Colab instance into a **Distributed Worker** for the Flaw Chess Engine.

### 🏁 Setup Options:
Run **ONLY ONE** of the setup options below.

## 1.1 Option A: Setup via GitHub (Recommended)
Use this if you have pushed your code to GitHub.

In [ ]:
GITHUB_URL = "https://github.com/Gideon-gg101/flaw-engine.git"

# 1. Clean up everything
!rm -rf /content/flaw
print("Starting Clone...")

# 2. Clone from GitHub
!git clone {GITHUB_URL} /content/flaw

# 3. Check what was created
if os.path.exists('/content/flaw'):
    print("\n✅ GitHub Sync Successful!")
    %cd /content/flaw
else:
    print("\n❌ FAILED to clone. Ensure your repository is PUBLIC!")

## 1.2 Option B: Setup via Manual ZIP
Use this if GitHub is giving you path errors. Upload `flaw_cloud_bundle.zip` to the file explorer on the left.

In [ ]:
!rm -rf /content/flaw
!mkdir -p /content/flaw
!unzip -o /content/flaw_cloud_bundle.zip -d /content/flaw

import os
if os.path.exists('/content/flaw'):
    print("✅ Unzip Successful!")
    %cd /content/flaw
else:
    print("❌ FAILED to find ZIP. Did you upload 'flaw_cloud_bundle.zip'?")

## 2. Environment & Build
Run this to compile the engine.

In [ ]:
!apt-get install -y cmake g++ > /dev/null
!pip install requests torch > /dev/null

import os
if not os.path.exists('/content/flaw'):
    raise Exception("❌ Stop! Run Section 1 first.")

%cd /content/flaw
!mkdir -p core/build
%cd core/build
!cmake ..
!make -j$(nproc)

# Move the built library to root AND ai folder for robust importing
print("\nMoving library to root and ai/ folder...")
!find . -name "flaw_core*.so" -exec cp {} /content/flaw/ \;
!find . -name "flaw_core*.so" -exec cp {} /content/flaw/ai/ \;

# Also create a simple symlink to simplify the name
%cd /content/flaw
!ln -sf flaw_core*.so flaw_core.so
!ln -sf flaw_core*.so ai/flaw_core.so

print("\n✅ Build Complete!")
!ls -F

## 3. Verify & Start Training
Replace the `MASTER_URL` with your local `ngrok` URL.

In [ ]:
MASTER_URL = "https://scabbardless-haywood-spangly.ngrok-free.dev"

import sys
sys.path.append('/content/flaw')

try:
    import flaw_core
    print("✅ C++ Core Loaded Successfully!")
except Exception as e:
    print(f"❌ ERROR: Still cannot find flaw_core: {e}")
    !find /content -name "flaw_core*"

# Export path and launch
!export PYTHONPATH=$PYTHONPATH:/content/flaw && python3 -m ai.selfplay_worker --master {MASTER_URL}